In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import sys
import cv2 as cv
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
combine_fold = 'C:/Users/oryan/Documents/mergers-in-cosmos/cats-to-combine'
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
fig_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/paper-source/figures'
cos_dat_fold = 'E:/cosmos-data'

In [3]:
df_cat = pd.read_csv(f'{combine_fold}/dor-all-restaged-full.csv', index_col = 0)

C:\Users\oryan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_cat

,ORyanID_23,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage4,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,860686,stage3,150.644522,2.237226,10047.978516,22713.351562,0.000051,0.000046,1.571297e-06,...,-9.314209,-9.106611,-8.941274,-8.713107,-8.541205,0.341270,0.545498,0.732327,0.906665,1.020098
2,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
3,4000705540034,1304538,stage3,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
4,4000705540064,1310001,stage3,149.701230,2.660343,32667.441406,32867.789062,0.000026,0.000020,-1.023530e-05,...,-9.882174,-9.855921,-9.816085,-9.697225,-9.573506,0.455258,0.482555,0.522239,0.600383,0.660279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176,4001204758614_extra,1520733,stage4,149.891374,2.866981,28107.876953,37826.437500,0.000086,0.000104,-9.313976e-07,...,-9.441206,-9.279022,-9.095134,-9.003501,-8.955496,0.120441,0.213251,0.306832,0.556012,0.731876
4177,4546055995_extra,1520749,stage2,150.383110,2.865367,16320.103516,37787.917969,0.000131,0.000236,7.123858e-05,...,-9.857880,-9.658215,-9.485625,-9.235816,-8.971795,0.162147,0.238640,0.432004,0.592193,0.687256
4178,4000927318133_extra,1523385,stage3,149.789910,2.865132,30540.179688,37782.648438,0.000045,0.000059,1.600747e-05,...,-9.525900,-9.472779,-9.353454,-9.273221,-9.213521,0.262676,0.355466,0.410356,0.489105,0.566003
4179,4000747939904_extra,1524909,stage3,150.450581,2.872371,14702.710938,37956.468750,0.000248,0.000292,7.378556e-06,...,-9.556987,-9.385500,-9.154130,-9.026810,-8.938077,0.153294,0.283249,0.436332,0.600615,0.693621


In [5]:
df_red_tmp = df_cat.query('ez_sfr < 10')
df_red = df_red_tmp.query('lp_mass_best > 9.25 and lp_mass_best <= 12.5')

### Cross Matching with AGN Catalogues

In [6]:
df_red

,ORyanID_23,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage4,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,860686,stage3,150.644522,2.237226,10047.978516,22713.351562,0.000051,0.000046,1.571297e-06,...,-9.314209,-9.106611,-8.941274,-8.713107,-8.541205,0.341270,0.545498,0.732327,0.906665,1.020098
3,4000705540034,1304538,stage3,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
4,4000705540064,1310001,stage3,149.701230,2.660343,32667.441406,32867.789062,0.000026,0.000020,-1.023530e-05,...,-9.882174,-9.855921,-9.816085,-9.697225,-9.573506,0.455258,0.482555,0.522239,0.600383,0.660279
5,4000705540104,1308927,stage3,149.688716,2.661469,32967.476562,32894.925781,0.000027,0.000027,-7.739382e-06,...,-8.919805,-8.915024,-8.908461,-8.900943,-8.896926,0.886180,0.922628,0.971381,1.015397,1.057468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176,4001204758614_extra,1520733,stage4,149.891374,2.866981,28107.876953,37826.437500,0.000086,0.000104,-9.313976e-07,...,-9.441206,-9.279022,-9.095134,-9.003501,-8.955496,0.120441,0.213251,0.306832,0.556012,0.731876
4177,4546055995_extra,1520749,stage2,150.383110,2.865367,16320.103516,37787.917969,0.000131,0.000236,7.123858e-05,...,-9.857880,-9.658215,-9.485625,-9.235816,-8.971795,0.162147,0.238640,0.432004,0.592193,0.687256
4178,4000927318133_extra,1523385,stage3,149.789910,2.865132,30540.179688,37782.648438,0.000045,0.000059,1.600747e-05,...,-9.525900,-9.472779,-9.353454,-9.273221,-9.213521,0.262676,0.355466,0.410356,0.489105,0.566003
4179,4000747939904_extra,1524909,stage3,150.450581,2.872371,14702.710938,37956.468750,0.000248,0.000292,7.378556e-06,...,-9.556987,-9.385500,-9.154130,-9.026810,-8.938077,0.153294,0.283249,0.436332,0.600615,0.693621


In [7]:
def get_redshift(ez_z, lp_z):
    if np.isnan(ez_z):
        if not np.isnan(lp_z):
            return lp_z
        else:
            return np.nan
    
    if np.isnan(lp_z):
        if not np.isnan(ez_z):
            return ez_z
        else:
            return np.nan
    
    if lp_z < ez_z:
        return lp_z
    elif ez_z <= lp_z:
        return ez_z
    else:
        return 'what'

In [8]:
df_redshifts = df_red.assign(redshift = df_red.apply(lambda row: get_redshift(row.ez_z_phot,row.lp_zBEST), axis = 1))

In [9]:
df_vol_lim = df_redshifts[['ORyanID_23', 'ID', 'redshift', 'ALPHA_J2000', 'DELTA_J2000']].query('redshift <= 1.2')

In [10]:
len(df_vol_lim)

3378

In [37]:
df_coords = (
    df_cat[['ID', 'ALPHA_J2000', 'DELTA_J2000']]
    .rename(columns = {'ALPHA_J2000':'RA','DELTA_J2000':'DEC'})
    .set_index('ID')
)

In [38]:
coords_dict = df_coords.to_dict(orient='index')

### Smolcic Catalogue 

In [39]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_smolcic_et_al.fits') as hdul:
    smol_rec = hdul[1].data

In [40]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_smolcic_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA_J2000'] * u.deg, dec = tab['DEC_VLA_J2000'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'smol_ID' : tab[min_ind]['ID_VLA'], 'RA' : tab[min_ind]['RA_VLA_J2000'], 'Dec' : tab[min_ind]['DEC_VLA_J2000']}

In [41]:
def matching_smolcic_ids(ra, dec, smol_rec):
    arcsec_sep_deg = 10 / (60 * 60)
    
    matching_tab = smol_rec[(smol_rec['RA_VLA_J2000'] > ra - arcsec_sep_deg) & (smol_rec['RA_VLA_J2000'] < ra + arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] > dec - arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'smol_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'smol_ID' : matching_tab['ID_VLA'][0], 'RA' : matching_tab['RA_VLA_J2000'][0], 'Dec' : matching_tab['DEC_VLA_J2000'][0]}
    elif len(matching_tab) > 1:
        return get_correct_smolcic_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [42]:
smolcic_matched_dict = {}

In [43]:
done_ids = list(smolcic_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    smolcic_matched_dict[i] = matching_smolcic_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], smol_rec)

100%|██████████| 4181/4181 [00:27<00:00, 153.48it/s]


In [44]:
smolcic_matched_df = pd.DataFrame.from_dict(smolcic_matched_dict, orient = 'index').dropna().reset_index().rename(columns = {'index' : 'ID', 0 : 'smol_id'})

In [45]:
smolcic_matched_df

,ID,smol_ID,RA,Dec
0,1264241,1766.0,150.321804,2.617930
1,844303,350.0,150.059144,2.219995
2,219965,3890.0,150.224529,1.618718
3,230625,10131.0,150.231835,1.627988
4,246486,8866.0,150.233316,1.651868
...,...,...,...,...
1207,1519437,2509.0,150.151975,2.860138
1208,1519536,1517.0,150.380343,2.865278
1209,1520749,1517.0,150.380343,2.865278
1210,1524909,8321.0,150.448344,2.875046


In [46]:
def dedup_smol(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return {'ID' : int(df.ID.iloc[0]), 'smol_ID' : df.smol_ID.iloc[0], 'smol_RA' : df.RA.iloc[0], 'smol_Dec' : df.Dec.iloc[0]}
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return {'ID' : int(min_row.ID),'smol_ID' : min_row.smol_ID.iloc[0], 'smol_RA' : min_row.RA.iloc[0], 'smol_Dec' : min_row.Dec.iloc[0]}

In [47]:
df_smol_merg = (
    smolcic_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'smol_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [60]:
smol_dedup_dict = df_smol_merg.groupby('smol_ID').apply(lambda df: dedup_smol(df)).to_dict()

In [61]:
smol_dedup_df = pd.DataFrame.from_dict(smol_dedup_dict, orient = 'index').reset_index().drop(columns = ['index'])

In [62]:
def getting_smol_info(smol_id, smol_rec):
    row = smol_rec[smol_rec['ID_VLA'] == smol_id]
    clsfs = [row['SFG'][0], row['Clean_SFG'][0], row['Radio_excess'][0]]
    
    if clsfs[0] == 'true' and clsfs[1] == 'true':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'false':
        return 'AGN'
    elif clsfs[0] == 'true' and clsfs[1] == 'false':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'true':
        print('Something wrong: 1')
        sys.exit()
    elif clsfs[3] == 'true':
        return 'Radio_Excess'

In [96]:
df_tmp_seps = smol_dedup_df.merge(df_cat[['ID', 'ALPHA_J2000', 'DELTA_J2000']], on = 'ID', how = 'left')

In [97]:
def calc_sep(cos_ra, cos_dec, agn_ra, agn_dec):
    cos_coord = SkyCoord(ra = cos_ra * u.deg, dec = cos_dec * u.deg, frame = 'fk5')
    agn_coord = SkyCoord(ra = agn_ra * u.deg, dec = agn_dec * u.deg, frame = 'fk5')
    sep_arcsec = (cos_coord.separation(agn_coord)).to(u.arcsec).value
    return sep_arcsec

In [98]:
df_seps = (
    df_tmp_seps
    .assign(seps = df_tmp_seps.apply(lambda row: calc_sep(row.ALPHA_J2000, row.DELTA_J2000, row.smol_RA, row.smol_Dec), axis = 1))
)

In [99]:
df_smol_sep_acc = df_seps[['ID', 'smol_ID']]#df_seps.query('seps <= 1')[['ID', 'smol_ID']]

In [100]:
smol_clsf_dict = {}

In [101]:
smol_dedup_dict = df_smol_sep_acc.set_index('ID').to_dict(orient = 'index')

In [102]:
done_ids = list(smol_clsf_dict.keys())
for i in tqdm(list(smol_dedup_dict.keys())):
    if i in done_ids:
        continue
    
    smol_clsf_dict[i] = getting_smol_info(smol_dedup_dict[i]['smol_ID'], smol_rec)

100%|██████████| 802/802 [00:05<00:00, 159.83it/s]


In [103]:
smol_clsf_df = pd.DataFrame.from_dict(smol_clsf_dict, orient = 'index').reset_index().rename(columns = {'index' : 'ID', 0 : 'agn_clsf'})

In [104]:
len(smol_clsf_df)

802

In [105]:
smol_clsf_df.drop_duplicates(subset='ID', keep='first')

,ID,agn_clsf
0,404744,AGN
1,835729,AGN
2,1152835,AGN
3,1095588,AGN
4,1026253,AGN
...,...,...
797,1081104,SFG
798,997145,SFG
799,1341925,SFG
800,893170,AGN


In [106]:
smol_clsf_df.agn_clsf.value_counts()

SFG    551
AGN    251
Name: agn_clsf, dtype: int64

In [107]:
smol_cat_ids = list(smol_clsf_df.ID)

### The VLA 3GHz Counterpart Array

In [108]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_delvecchio_et_al.fits') as hdul:
    vla_cat = hdul[1].data

In [109]:
vla_cat.names

['ID_VLA3',
 'RA_VLA3',
 'DEC_VLA3',
 'Z_BEST',
 'Z_TYPE',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'flag_herschel3',
 'Mstar',
 'SFR_IR',
 'NUV_R',
 'XRAY_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Radio_excess',
 'Class']

In [110]:
len(vla_cat)

7729

In [111]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_vla_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA3'] * u.deg, dec = tab['DEC_VLA3'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'VLA_ID' : tab[min_ind]['ID_VLA3'], 'RA' : tab[min_ind]['RA_VLA3'], 'Dec' : tab[min_ind]['DEC_VLA3']}

In [112]:
def matching_vla_ids(ra, dec, vla_cat):
    arcsec_sep_deg = 10 / (60 * 60)
    
    matching_tab = vla_cat[(vla_cat['RA_VLA3'] > ra - arcsec_sep_deg) & (vla_cat['RA_VLA3'] < ra + arcsec_sep_deg) & (vla_cat['DEC_VLA3'] > dec - arcsec_sep_deg) & (vla_cat['DEC_VLA3'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'VLA_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'VLA_ID' : matching_tab['ID_VLA3'][0], 'RA' : matching_tab['RA_VLA3'][0], 'Dec' : matching_tab['DEC_VLA3'][0]}
    elif len(matching_tab) > 1:
        return get_correct_vla_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [113]:
vla_matched_dict = {}

In [114]:
done_ids = list(vla_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    vla_matched_dict[i] = matching_vla_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], vla_cat)

100%|██████████| 4181/4181 [00:21<00:00, 196.08it/s]


In [115]:
vla_matched_df = (
    pd.DataFrame.from_dict(vla_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID', 0 : 'VLA_ID_arr'})
)

In [116]:
vla_matched_df

,ID,VLA_ID,RA,Dec
0,1264241,1766.0,150.321804,2.617930
1,844303,350.0,150.059144,2.219995
2,246486,8866.0,150.233316,1.651868
3,250320,8866.0,150.233316,1.651868
4,1074532,3843.0,149.938835,2.433375
...,...,...,...,...
1090,1519437,2509.0,150.151975,2.860138
1091,1519536,1517.0,150.380343,2.865278
1092,1520749,1517.0,150.380343,2.865278
1093,1524909,8321.0,150.448344,2.875046


In [117]:
def dedup_vla(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return {'ID' : int(df.ID.iloc[0]),'VLA_ID' : df.VLA_ID.iloc[0], 'VLA_RA' : df.RA.iloc[0], 'VLA_Dec' : df.Dec.iloc[0]}
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return {'ID' : int(min_row.ID), 'VLA_ID' : min_row.VLA_ID.iloc[0], 'VLA_RA' : min_row.RA.iloc[0], 'VLA_Dec' : min_row.Dec.iloc[0]}

In [118]:
df_vla_merg = (
    vla_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'VLA_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [119]:
vla_dedup_dict = df_vla_merg.groupby('VLA_ID').apply(lambda df: dedup_vla(df)).to_dict()

In [120]:
vla_dedup_df = pd.DataFrame.from_dict(vla_dedup_dict, orient = 'index')
vla_dedup_df

,ID,VLA_ID,VLA_RA,VLA_Dec
47.0,404744,47.0,150.407410,1.803237
61.0,835729,61.0,149.649621,2.209249
66.0,1152835,66.0,150.022658,2.508060
68.0,1095588,68.0,149.485493,2.454786
84.0,1026253,84.0,149.952839,2.386293
...,...,...,...,...
10808.0,1081104,10808.0,149.641496,2.442626
10838.0,997145,10838.0,149.471313,2.360339
10869.0,1341925,10869.0,150.062132,2.696190
10912.0,893170,10912.0,149.508755,2.261351


In [121]:
vla_checked = (
    vla_dedup_df
    .assign(in_smol = vla_dedup_df.ID.apply(lambda x: 'y' if x in smol_cat_ids else 'n'))
)

vla_checked_df = vla_checked.query('in_smol == "n"').drop(columns = 'in_smol')

In [122]:
len(vla_checked_df)

1

In [123]:
vla_checked = (
    vla_checked_df
    .merge(df_smol_merg[['ID', 'ALPHA_J2000', 'DELTA_J2000']], on = 'ID', how = 'left')
)

In [124]:
vla_checked

,ID,VLA_ID,VLA_RA,VLA_Dec,ALPHA_J2000,DELTA_J2000
0,715417,1804.0,149.893611,2.10777,149.896032,2.107181


In [125]:
vla_seps = (
    vla_checked
    .assign(seps = vla_checked.apply(lambda row: calc_sep(row.ALPHA_J2000, row.DELTA_J2000, row.VLA_RA, row.VLA_Dec), axis = 1))
)

In [126]:
vla_dict =  vla_seps[['ID', 'VLA_ID']].set_index('ID').to_dict(orient = 'index') #vla_seps.query('seps <= 1')[['ID', 'VLA_ID']].set_index('ID').to_dict(orient = 'index')

In [127]:
vla_seps

,ID,VLA_ID,VLA_RA,VLA_Dec,ALPHA_J2000,DELTA_J2000,seps
0,715417,1804.0,149.893611,2.10777,149.896032,2.107181,8.962658


In [128]:
len(vla_dict)

1

In [129]:
vla_cat_dict = {}

In [130]:
def getting_vla_info(vla_id, vla_cat):
    row = vla_cat[vla_cat['ID_VLA3'] == vla_id]
    ans = row['CLASS'][0]
    
    if len(ans) < 0.5:
        return 'SFG'
    else:
        return ans

In [131]:
done_ids = list(vla_cat_dict.keys())
for i in tqdm(list(vla_dict.keys())):
    if i in done_ids:
        continue
    
    vla_cat_dict[i] = getting_vla_info(vla_dict[i]['VLA_ID'], vla_cat)

100%|██████████| 1/1 [00:00<00:00, 179.88it/s]


In [132]:
vla_cat_df = pd.DataFrame.from_dict(vla_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
vla_cat_df

,clsf
715417,HLAGN


In [133]:
vla_cat_ids = list(vla_cat_df.reset_index()['index'])

### Applying to Chandra

In [134]:
with fits.open(f'{cos_dat_fold}/chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul:
    chandra_cat = hdul[1].data

In [135]:
len(chandra_cat)

4016

In [136]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_chandra_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    chandra_coords = SkyCoord(ra = tab['RA_x'] * u.deg, dec = tab['DEC_x'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(chandra_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'CHANDRA_ID' : tab[min_ind]['id_x'], 'RA' : tab[min_ind]['RA_x'], 'Dec' : tab[min_ind]['DEC_x']}

In [137]:
def matching_chandra_ids(ra, dec, chandra_cat):
    arcsec_sep_deg = 10 / (60 * 60)
    
    matching_tab = chandra_cat[(chandra_cat['RA_x'] > ra - arcsec_sep_deg) & (chandra_cat['RA_x'] < ra + arcsec_sep_deg) & (chandra_cat['DEC_x'] > dec - arcsec_sep_deg) & (chandra_cat['DEC_x'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'CHANDRA_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'CHANDRA_ID' : matching_tab['id_x'][0], 'RA' : matching_tab['RA_x'][0], 'Dec' : matching_tab['DEC_x'][0]}
    elif len(matching_tab) > 1:
        return get_correct_chandra_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [138]:
chandra_matched_dict = {}

In [139]:
done_ids = list(chandra_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_matched_dict[i] = matching_chandra_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], chandra_cat)

100%|██████████| 4181/4181 [00:59<00:00, 70.09it/s]


In [140]:
chandra_matched_df = (
    pd.DataFrame.from_dict(chandra_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID'})
)

In [141]:
chandra_matched_df

,ID,CHANDRA_ID,RA,Dec
0,1264241,lid_2035,150.320007,2.620354
1,844303,cid_1139,150.059130,2.219824
2,230625,cid_737,150.231813,1.628142
3,525757,cid_236,150.027300,1.924484
4,525280,cid_799,149.912110,1.923509
...,...,...,...,...
418,1462750,lid_3224,150.673890,2.814361
419,1464002,lid_3224,150.673890,2.814361
420,1464904,lid_3181,150.379151,2.817332
421,1465158,lid_3224,150.673890,2.814361


In [142]:
df_chandra_merge = (
    chandra_matched_df
    .merge(df_red[['ID','ALPHA_J2000','DELTA_J2000']], on = 'ID', how = 'left')
)

In [143]:
df_chandra_seps = (
    df_chandra_merge
    .assign(seps = df_chandra_merge.apply(lambda row: calc_sep(row.RA, row.Dec, row.ALPHA_J2000, row.DELTA_J2000), axis = 1))
)

In [156]:
df_chandra_seps_rem = df_chandra_seps.dropna().query('seps <= 5')

In [158]:
len(df_chandra_seps_rem)

157

In [159]:
df_chandra_seps_rem

,ID,CHANDRA_ID,RA,Dec,ALPHA_J2000,DELTA_J2000,seps
1,844303,cid_1139,150.059130,2.219824,150.059115,2.219933,0.398189
2,230625,cid_737,150.231813,1.628142,150.231845,1.627975,0.611797
4,525280,cid_799,149.912110,1.923509,149.912175,1.923344,0.638883
7,349071,lid_1008,149.867547,1.747775,149.867750,1.747759,0.730922
12,858361,cid_405,149.779040,2.232796,149.779096,2.232736,0.294004
...,...,...,...,...,...,...,...
408,1236245,cid_614,149.991774,2.589941,149.991115,2.590308,2.712790
409,1272183,cid_726,150.129825,2.624839,150.129809,2.624708,0.474261
413,1312891,lid_3112,150.402030,2.666955,150.401715,2.666226,2.857708
419,1464002,lid_3224,150.673890,2.814361,150.674601,2.814980,3.391310


In [160]:
def dedup_chandra(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [161]:
chandra_dedup_df = pd.DataFrame(df_chandra_seps_rem.groupby('CHANDRA_ID').apply(lambda df: dedup_chandra(df))).reset_index().rename(columns = {0 : 'ID'})

In [162]:
len(chandra_dedup_df)

118

In [163]:
chandra_checked_df = (
    chandra_dedup_df
    .assign(in_cmol = chandra_dedup_df.ID.apply(lambda x: 'y' if x in smol_cat_ids else 'n'))
    # .assign(in_vla = chandra_dedup_df.ID.apply(lambda x: 'y' if x in vla_cat_ids else 'n'))
)

In [164]:
len(chandra_checked_df)

118

In [165]:
chandra_mpa_checked = chandra_checked_df.query('in_cmol == "n"')

In [166]:
len(chandra_mpa_checked)

49

In [167]:
chandra_dict = chandra_mpa_checked.set_index('ID').to_dict(orient = 'index')

In [168]:
chandra_cat_dict = {}

In [169]:
def getting_chandra_info(chandra_id, chandra_cat):
    row = chandra_cat[chandra_cat['id_x'] == chandra_id]    
    hard_lum = row['Lx_210'][0]
    
    if hard_lum < 42:
        return 'SFG'
    elif hard_lum >= 42:
        return 'AGN'
    else:
        return np.nan

In [170]:
done_ids = list(chandra_cat_dict.keys())
for i in tqdm(list(chandra_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_cat_dict[i] = getting_chandra_info(chandra_dict[i]['CHANDRA_ID'], chandra_cat)

100%|██████████| 49/49 [00:00<00:00, 72.58it/s]


In [171]:
chandra_cat_df = pd.DataFrame.from_dict(chandra_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
chandra_cat_df

,clsf
352373,AGN
788036,AGN
649583,AGN
1281486,AGN
735085,AGN
806643,AGN
1010949,AGN
1070007,AGN
872909,AGN
927713,AGN


In [172]:
chandra_cat_df.clsf.value_counts()

AGN    38
SFG    11
Name: clsf, dtype: int64

### Putting it all Together!

chandra_agn_cat

vla_cat_df

clsf_df

smol_clsf_df

In [173]:
# vla_cat_comb = vla_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf' })

In [198]:
df_agn_cat = pd.concat([
    # vla_cat_comb,
    smol_clsf_df, 
    # chandra_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf'})
# df_agn_cat = pd.concat([vla_cat_comb, smol_clsf_df])
])

In [199]:
len(df_agn_cat)

802

In [200]:
df_agn_cat

,ID,agn_clsf
0,404744,AGN
1,835729,AGN
2,1152835,AGN
3,1095588,AGN
4,1026253,AGN
...,...,...
797,1081104,SFG
798,997145,SFG
799,1341925,SFG
800,893170,AGN


In [201]:
df_agn_cat.agn_clsf.value_counts()

SFG    551
AGN    251
Name: agn_clsf, dtype: int64

In [202]:
len(df_agn_cat.drop_duplicates(subset='ID', keep='first'))

802

In [203]:
df_agn_dedup = df_agn_cat.drop_duplicates(subset='ID', keep='first')

In [204]:
df_agn_dedup.agn_clsf.value_counts()

SFG    551
AGN    251
Name: agn_clsf, dtype: int64

In [205]:
df_agn_dedup.to_csv(f'{combine_fold}/source-agn-cat-fixed.csv')